In [ ]:
import os
import openai
import asyncio
import nest_asyncio
import edge_tts
from moviepy.video.io.VideoFileClip import VideoFileClip,AudioFileClip




from dotenv import load_dotenv
load_dotenv()
# Set your OpenAI API key (you can also set it via environment variables)
openai_api_key = os.getenv("OPENAI_API_KEY")

# ---------------------------------------------------------------------------
# STEP 1: Translate and Improve the Transcript with OpenAI's LLM
# ---------------------------------------------------------------------------
def improve_translation(transcript: str) -> str:
    """
    Translate the given transcript into natural, fluent French and improve its clarity.
    """
    prompt = (
        "Please translate the following transcript into natural, fluent French. "
        "Improve the translation for clarity and natural tone:\n\n" + transcript
    )
    
    response = openai.ChatCompletion.create(
        model="gpt-4o",  # or "gpt-3.5-turbo" if GPT-4 is unavailable
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3 # Adjust temperature for creativity vs. accuracy
    )
    
    # Extract the translated content from the response
    translated_text = response.choices[0].message["content"].strip()
    return translated_text

# ---------------------------------------------------------------------------
# STEP 2: Generate French Voice-Over Audio Using Edge-TTS
# ---------------------------------------------------------------------------
async def generate_french_audio(text: str, output_audio: str = "french_voice_over.mp3"):
    """
    Generate French speech audio using Edge-TTS with the desired voice and rate.
    """
    # French voice and speech rate settings
    voice = "fr-BE-GerardNeural"  # Change if needed
    rate = "-10%"  # Slightly slower for clarity

    # Create the TTS communicator and save the audio file
    communicator = edge_tts.Communicate(text, voice, rate=rate)
    await communicator.save(output_audio)

def run_generate_audio(text: str, output_audio: str = "french_voice_over.mp3"):
    """
    Run the asynchronous TTS generation using nest_asyncio to avoid event loop issues.
    """
    nest_asyncio.apply()  # Allows asyncio to run in environments like Jupyter
    loop = asyncio.get_event_loop()
    loop.run_until_complete(generate_french_audio(text, output_audio))

# ---------------------------------------------------------------------------
# STEP 3: Merge the New French Audio with the Original Video
# ---------------------------------------------------------------------------
def merge_audio_with_video(video_path: str, audio_path: str, output_video: str = "final_video.mp4"):
    """
    Replace the original audio in the video with the new French voice-over.
    """
    video_clip = VideoFileClip(video_path)
    audio_clip = AudioFileClip(audio_path)
    # Set the new audio to the video clip
     # final_video = video_clip.set_audio(audio_clip)
    final_video = video_clip.with_audio(audio_clip)

    # Write the final video to file (adjust codec and audio settings as needed)
    final_video.write_videofile(output_video, codec="libx264", audio_codec="aac")


import re

def remove_timestamps(transcript: str) -> str:
    """
    Remove common timestamp patterns (e.g., '0:00:', '1:23:', or 'minuit') from the transcript.
    Adjust the regex pattern if your timestamps have a different format.
    """
    # This regex matches timestamps like "0:00:" or "1:23:" at the beginning of lines
    cleaned_lines = []
    for line in transcript.splitlines():
        # Remove timestamp pattern at start of the line
        cleaned_line = re.sub(r'^\s*\d{1,2}:\d{2}:\s*', '', line)
        # Optionally remove occurrences of the word "minuit" if they are only meant for timing
        cleaned_line = re.sub(r'\bminuit\b', '', cleaned_line, flags=re.IGNORECASE)
        cleaned_lines.append(cleaned_line.strip())
    return "\n".join(cleaned_lines)






# ---------------------------------------------------------------------------
# MAIN EXECUTION
# ---------------------------------------------------------------------------
if __name__ == "__main__":
    # Original transcript text with timestamps (as provided)
    transcript_text = """0:00: In this demo, we will explore about navigation flow. 
0:06: And how we can customize and assign provisions to user groups. 
0:12: Navigation flow enhances user experience with intuitive structured pathways for effortless navigation across modules and tasks. 
0:24: Enables smooth transition between data entry, reporting, and process management, optimizing efficiency within planning and analytics workflows. 
0:38: Once a user has logged into EPM planning application, they will see this landing page. 
0:47: Clicking on the 4 dots on the right corner, you will see. 
0:54: All the active navigation flows for this application. 
0:59: EPM planning comes with default navigation flow. 
1:06: You can toggle to different navigation flow simply by clicking on it. 
1:13: Let's click on EBM cloud navigation flow. 
1:18: You can see all the cards and cluster enabled for this particular navigation floor. 
1:27: Now let's toggle back to the SM flow navigation flow. 
1:35: And we will take a look at how we can customize the SM navigation flow. 
1:42: You can see on the screen we have 6 cards and cluster. 
1:48: Cluster is group of cuts. 
1:54: And we are going to enable a new cluster called Financials, and we're going to disable the cluster strategic modeling in this demo. 
2:08: To configure navigation flow, a user needs to have service admin access. 
2:16: Now clicking on tools. 
2:20: You will see the cluster navigation flow. 
2:25: Here you can see multiple navigation flows configured and some of them are active. 
2:34: And some of them are inactive. 
2:39: You can select a navigation flow. 
2:43: And clicking on the action gear on the right corner. 
2:49: We'll let you to copy this navigation flow. 
2:54: Delete and validate. 
3:00: Before modifying any navigation flow, we need to make sure the navigation flow is inactive. 
3:09: So clicking on it will make it inactive. 
3:14: To configure the navigation flow. 
3:18: I'm going to click on the SM navigation flow name. 
3:26: And we can start making changes here. 
3:29: Once I have clicked and navigated to this flow, and here you can see the type where it defines the cluster. 
3:41: Or it's a card? 
3:43: And the state of visibility enable or disabled. 
3:51: The order which you can sort. 
3:54: And the option to delete. 
3:58: From this navigation. 
4:01: For this demo, let's enable financials. 
4:07: We're going to expand financials, so financials is a cluster, and it will have multiple cards. 
4:16: I'm going to move this car up because I want to see revenue first. 
4:25: And I'm going to collapse the final shells. 
4:30: Now, there is strategic modeling enabled, I'm going to disable this. 
4:36: And also, I have the permission to assign to a particular group. 
4:44: Or Particular user role. 
4:47: And this will allow only users belonging to those groups or roles to access this navigation flow. 
4:57: So it helps in maintaining security and limiting departments from accessing these financial modules. 
5:07: are going to save and close. 
5:13: And once I'm done with all the changes, I will have to activate this back, so clicking on it will activate the navigation flow. 
5:23: Now to refresh the flow, I'm going to go to the homepage where I'm still seeing the old view. 
5:30: I don't see the financial card and I'm seeing the strategy which I should not be seeing, so I need to go to the 4 dots and refresh the navigation flow from this refresh button. 
5:47: Once I do that, you can see I have a new card financials. 
5:54: I won't see the strategic modeling card. 
5:56: And Clicking on the financial cluster, you can see multiple cards and you can see revenue as we moved it to the top to display on the first. 
6:09: So let me click on revenue, and it lost the revenue for your FY 23, and you can see consolidated financial data in multiple graphical views, and you can see how the data is retrieved within seconds. 
6:28: We can hover over these options to see more details for different graphical views with all the data. 
6:39: You can notice that we have multiple tabs, and clicking on those tabs take us to those views with more detailed revenue details. 
6:50: There are also tabs on the top. 
6:53: And the taps on the bottom and just clicking on them will take you to that particular dashboard. 
7:02: Hovering over the graphical view. 
7:06: You can see the more detailed information. 
7:12: In this demo we explored our nation flow and how we can customize and assign permissions to user groups."""
    
    
    # Remove the timestamps
    clean_transcript = remove_timestamps(transcript_text)
    print("Clean Transcript:\n", clean_transcript)
        # -----------------------------------------------------------------------
    # Translate the transcript into natural, fluent French
    print("Translating transcript to French...")
    french_translation = improve_translation(clean_transcript)
    print("French translation obtained.")
    
    # Save the translated transcript to a text file (optional)
    with open("french_transcript.txt", "w", encoding="utf-8") as file:
        file.write(french_translation)
    
    # -----------------------------------------------------------------------
    # Generate French voice-over audio from the improved translation
    print("Generating French voice audio...")
    run_generate_audio(french_translation, output_audio="french_voice_over.mp3")
    print("French voice audio generated as 'french_voice_over.mp3'.")
    
    # -----------------------------------------------------------------------
    # Merge the generated French audio with the original video
    original_video = "Navigation_Flow_COM.mp4"  # Replace with your MP4 video file path
    print("Merging the French audio with the original video...")
    merge_audio_with_video(original_video, "french_voice_over.mp3", output_video="Navigation_Flow_COM_French.mp4")
    print("Final video saved as 'final_video.mp4'.")


Translating transcript to French...
French translation obtained.
Generating French voice audio...
French voice audio generated as 'french_voice_over.mp3'.
Merging the French audio with the original video...
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41', 'encoder': 'https://clipchamp.com', 'comment': 'Create videos with https://clipchamp.com/en/video-editor - free online video editor, video compressor, video converter.'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [3840, 2160], 'bitrate': 4947, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(Main)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 48000, 'bitrate': 192, 'metadata': {'Metadata': '', 'handler_n

AttributeError: 'VideoFileClip' object has no attribute 'set_audio'

Résumé du code :
Ce script transforme une vidéo anglophone en version française en générant une piste audio synchronisée via synthèse vocale (TTS). Il traduit d'abord les sous-titres avec OpenAI (optionnel), ajuste les durées des segments audio pour correspondre à la vidéo, puis fusionne le tout avec la vidéo originale.

Principales librairies :

edge_tts : Génère la voix française synthétique (SylvieNeural) pour chaque segment de texte.

openai : Traduit/améliore le texte en français via GPT-4/3.5 (optionnel).

pydub : Manipule les fichiers audio (découpage, ajout de silence, concaténation).

moviepy : Fusionne l'audio final avec la vidéo originale.

asyncio/nest_asyncio : Gère les appels asynchrones pour la synthèse vocale.

Étapes clés :

Configuration :

Charge la clé OpenAI depuis .env.

Définit les paramètres de voix (débit, voix FR) et les chemins des fichiers.

Parsing du transcript :

Extrait les segments texte + timestamps avec une regex (0:00: Texte...).

Traduction (optionnelle) :

Utilise GPT-4/3.5 pour traduire/améliorer le texte en français naturel.

Génération audio :

Crée un fichier MP3 par segment via edge_tts (asynchrone).

Ajuste les durées : ajoute du silence ou tronque pour synchroniser avec la vidéo.

Fusion finale :

Concatène tous les segments audio en un fichier unique.

Remplace la piste audio de la vidéo originale avec moviepy.

Cas d'usage typique :
Localiser une vidéo tutorielle en français en conservant le timing original entre l'audio et les visuels, avec une voix synthétique personnalisée.



In [ ]:
import re
import os
import asyncio
import nest_asyncio
import openai
import edge_tts
from pydub.utils import which
from pydub import AudioSegment
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.audio.io.AudioFileClip import AudioFileClip
from dotenv import load_dotenv
load_dotenv()

# --- CONFIGURATION ---

# Set your OpenAI API key (if using translation/improvement)
openai.api_key = os.getenv("OPENAI_API_KEY")

# TTS settings for French voice
FRENCH_VOICE = "fr-CA-SylvieNeural"  # Adjust if needed
FRENCH_RATE = "-10%"  # Slightly slower for clarity

# Original video file path
VIDEO_PATH = "Navigation_Flow_COM.mp4"  # Replace with your MP4 video file
OUTPUT_VIDEO = "Navigation_Flow_COM_French.mp4"

# Final concatenated audio file name
FINAL_AUDIO_FILE = "final_french_voice.mp3"

# Temporary folder to store audio segments
SEGMENTS_DIR = "segments_temp"
os.makedirs(SEGMENTS_DIR, exist_ok=True)

# --- Set FFmpeg Path ---

# Manually set the path to FFmpeg to ensure the correct version is used
os.environ["PATH"] = r"C:\ffmpeg\bin" + ";" + os.environ["PATH"]

# Ensure that the path is correctly detected
ffmpeg_path = which("ffmpeg")
print(f"FFmpeg found at: {ffmpeg_path}")


# --- FUNCTIONS ---

def parse_transcript(transcript: str):
    """
    Parse a transcript with timestamps in the format 'mm:ss: text' (one per line)
    and return a list of segments: (start_time_seconds, text)
    """
    segments = []
    # Regex pattern for lines like "0:00: In this demo..."
    pattern = r'^\s*(\d{1,2}):(\d{2}):\s*(.*)$'
    for line in transcript.splitlines():
        match = re.match(pattern, line)
        if match:
            minutes, seconds, text = match.groups()
            start_time = int(minutes) * 60 + int(seconds)
            segments.append((start_time, text.strip()))
    return segments

def improve_translation(text: str) -> str:
    """
    (Optional) Use OpenAI's LLM to translate/improve the text into natural French.
    If you don't need this step, simply return the original text.
    """
    prompt = (
        "Please translate the following text into natural, fluent French, "
        "improving the wording for clarity:\n\n" + text
    )
    response = openai.ChatCompletion.create(
        model="gpt-4o",  # or "gpt-3.5-turbo"
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    translated_text = response.choices[0].message["content"].strip()
    return translated_text

async def generate_segment_audio(text: str, output_file: str, voice: str = FRENCH_VOICE, rate: str = FRENCH_RATE):
    """
    Asynchronously generate TTS audio for the given text segment using Edge-TTS.
    """
    communicator = edge_tts.Communicate(text, voice, rate=rate)
    await communicator.save(output_file)

def run_generate_audio_for_segment(text: str, output_file: str):
    """
    Run the asynchronous TTS generation for a segment.
    """
    nest_asyncio.apply()
    loop = asyncio.get_event_loop()
    loop.run_until_complete(generate_segment_audio(text, output_file))

def create_synchronized_audio(segments, final_output=FINAL_AUDIO_FILE):
    """
    Generate audio for each transcript segment, pad/truncate to match the intended duration,
    and concatenate them into one final audio file.
    
    Parameters:
      segments: list of tuples (start_time, text)
    """
    audio_segments = []
    num_segments = len(segments)
    
    for i, (start_time, text) in enumerate(segments):
        translated_text = improve_translation(text) # use translated_text = text if you don't want to use OpenAI for translation
        
        # Add validation for the translated text
        if not translated_text or len(translated_text.strip()) == 0:
            raise ValueError(f"Segment {i} (start: {start_time}s) has empty translated text!")
        
        print(f"\n--- Segment {i} (Start: {start_time}s) ---")
        print(f"Translated text: {translated_text}\n")  # <-- Critical for debugging
        
        segment_filename = os.path.join(SEGMENTS_DIR, f"segment_{i}.mp3")
        print(f"Generating audio for segment {i}...")
        run_generate_audio_for_segment(translated_text, segment_filename)
        segment_audio = AudioSegment.from_file(segment_filename)
        
        # Determine intended duration (if not the last segment, use difference to next segment's start time)
        if i < num_segments - 1:
            next_start = segments[i+1][0]
            intended_duration_ms = (next_start - start_time) * 1000
        else:
            intended_duration_ms = len(segment_audio)
        
        # Adjust the segment audio length:
        if len(segment_audio) < intended_duration_ms:
            # Append silence if audio is shorter than intended
            silence = AudioSegment.silent(duration=intended_duration_ms - len(segment_audio))
            segment_audio = segment_audio + silence
        else:
            # Truncate if longer
            segment_audio = segment_audio[:intended_duration_ms]
        
        # If this is the first segment and it doesn't start at time 0, add leading silence.
        if i == 0 and start_time > 0:
            leading_silence = AudioSegment.silent(duration=start_time * 1000)
            audio_segments.append(leading_silence)
        
        audio_segments.append(segment_audio)
    
    # Concatenate all segments
    final_audio = sum(audio_segments)
    final_audio.export(final_output, format="mp3")
    print(f"Final synchronized audio saved as {final_output}")
    return final_output

def merge_audio_with_video(video_path: str, audio_path: str, output_video: str = OUTPUT_VIDEO):
    """
    Merge the final French audio with the original video.
    """
    video_clip = VideoFileClip(video_path)
    audio_clip = AudioFileClip(audio_path)
    final_video = video_clip.with_audio(audio_clip)
    final_video.write_videofile(output_video, codec="libx264", audio_codec="aac")
    print(f"Final video saved as {output_video}")

# --- MAIN EXECUTION ---

if __name__ == "__main__":
    # Sample transcript with timestamps
    transcript_text = """0:00: In this demo, we will explore about navigation flow.
0:06: And how we can customize and assign provisions to user groups.
0:12: Navigation flow enhances user experience with intuitive structured pathways for effortless navigation across modules and tasks.
0:24: Enables smooth transition between data entry, reporting, and process management, optimizing efficiency within planning and analytics workflows.
0:38: Once a user has logged into EPM planning application, they will see this landing page.
0:47: Clicking on the 4 dots on the right corner, you will see.
0:54: All the active navigation flows for this application.
0:59: EPM planning comes with default navigation flow.
1:06: You can toggle to different navigation flow simply by clicking on it.
1:13: Let's click on EBM cloud navigation flow.
1:18: You can see all the cards and cluster enabled for this particular navigation flow.
1:27: Now let's toggle back to the SM flow navigation flow.
1:35: And we will take a look at how we can customize the SM navigation flow.
1:42: You can see on the screen we have 6 cards and cluster.
1:48: Cluster is group of cuts.
1:54: And we are going to enable a new cluster called Financials, and we're going to disable the cluster strategic modeling in this demo.
2:08: To configure navigation flow, a user needs to have service admin access.
2:16: Now clicking on tools.
2:20: You will see the cluster navigation flow.
2:25: Here you can see multiple navigation flows configured and some of them are active.
2:34: And some of them are inactive.
2:39: You can select a navigation flow.
2:43: And clicking on the action gear on the right corner.
2:49: We'll let you to copy this navigation flow.
2:54: Delete and validate.
3:00: Before modifying any navigation flow, we need to make sure the navigation flow is inactive.
3:09: So clicking on it will make it inactive.
3:14: To configure the navigation flow.
3:18: I'm going to click on the SM navigation flow name.
3:26: And we can start making changes here.
3:29: Once I have clicked and navigated to this flow, and here you can see the type where it defines the cluster.
3:41: Or it's a card?
3:43: And the state of visibility enable or disabled.
3:51: The order which you can sort.
3:54: And the option to delete.
3:58: From this navigation.
4:01: For this demo, let's enable financials.
4:07: We're going to expand financials, so financials is a cluster, and it will have multiple cards.
4:16: I'm going to move this card up because I want to see revenue first.
4:25: And I'm going to collapse the final shells.
4:30: Now, there is strategic modeling enabled, I'm going to disable this.
4:36: And also, I have the permission to assign to a particular group.
4:44: Or particular user role.
4:47: And this will allow only users belonging to those groups or roles to access this navigation flow.
4:57: So it helps in maintaining security and limiting departments from accessing these financial modules.
5:07: We're going to save and close.
5:13: And once I'm done with all the changes, I will have to activate this back, so clicking on it will activate the navigation flow.
5:23: Now to refresh the flow, I'm going to go to the homepage where I'm still seeing the old view.
5:30: I don't see the financial card and I'm seeing the strategy which I should not be seeing, so I need to go to the 4 dots and refresh the navigation flow from this refresh button.
5:47: Once I do that, you can see I have a new card financials.
5:54: I won't see the strategic modeling card.
5:56: And clicking on the financial cluster, you can see multiple cards and you can see revenue as we moved it to the top to display on the first.
6:09: So let me click on revenue, and it shows the revenue for your FY 23, and you can see consolidated financial data in multiple graphical views.
6:28: We can hover over these options to see more details for different graphical views with all the data.
6:39: You can notice that we have multiple tabs, and clicking on those tabs takes us to those views with more detailed revenue details.
6:50: There are also tabs on the top.
6:53: And the tabs on the bottom, and just clicking on them will take you to that particular dashboard.
7:02: Hovering over the graphical view.
7:06: You can see more detailed information.
7:12: In this demo we explored our navigation flow and how we can customize and assign permissions to user groups."""

    # --- Parse the transcript ---
    segments = parse_transcript(transcript_text)
    if not segments:
        print("No segments found. Please check your transcript format.")
        exit(1)
    
    # --- Create the synchronized audio track from segments ---
    final_audio_file = create_synchronized_audio(segments, final_output=FINAL_AUDIO_FILE)
    
    # --- Merge the generated French audio with the original video ---
    print("Merging the French audio with the original video...")
    merge_audio_with_video(VIDEO_PATH, final_audio_file, output_video=OUTPUT_VIDEO)
    print("Process completed successfully!")


FFmpeg found at: C:\ffmpeg\bin\ffmpeg.exe

--- Segment 0 (Start: 0s) ---
Translated text: Dans cette démonstration, nous allons explorer le flux de navigation.

Generating audio for segment 0...

--- Segment 1 (Start: 6s) ---
Translated text: Et comment nous pouvons personnaliser et attribuer des dispositions aux groupes d'utilisateurs.

Generating audio for segment 1...

--- Segment 2 (Start: 12s) ---
Translated text: Le flux de navigation améliore l'expérience utilisateur grâce à des parcours structurés et intuitifs, permettant une navigation sans effort à travers les modules et les tâches.

Generating audio for segment 2...

--- Segment 3 (Start: 24s) ---
Translated text: Permet une transition fluide entre la saisie de données, la création de rapports et la gestion des processus, optimisant ainsi l'efficacité des flux de travail en planification et en analyse.

Generating audio for segment 3...

--- Segment 4 (Start: 38s) ---
Translated text: Une fois qu'un utilisateur s'est connect

MoviePy - Done.
MoviePy - Writing video Navigation_Flow_COM_French.mp4



MoviePy - Done !
MoviePy - video ready Navigation_Flow_COM_French.mp4
Final video saved as Navigation_Flow_COM_French.mp4
Process completed successfully!


In [9]:
import os

segment_filename = os.path.join("segments_temp", "segment_0.mp3")

print(f"Checking if file exists: {segment_filename}")
print(os.path.exists(segment_filename))


Checking if file exists: segments_temp\segment_0.mp3
True


In [10]:
from pydub import AudioSegment

try:
    audio = AudioSegment.from_file("segments_temp/segment_0.mp3")
    print("Audio loaded successfully!")
except Exception as e:
    print(f"Error loading audio: {e}")


Error loading audio: [WinError 2] Le fichier spécifié est introuvable


In [11]:
segment_filename = "segments_temp/segment_0.mp3"
print(f"Path to file: {segment_filename}")
print(os.path.exists(segment_filename))  # Check if file exists


Path to file: segments_temp/segment_0.mp3
True


In [17]:
from pydub.utils import which
AudioSegment.ffmpeg = r"C:\ffmpeg\bin\ffmpeg.exe"  # Replace with your actual path to ffmpeg.exe

# Now try loading the audio again
try:
    audio = AudioSegment.from_mp3(r"C:\AI PROJECTS\video_transcript\segments_temp\segment_0.mp3")
    print("Audio loaded successfully!")
except Exception as e:
    print(f"Error loading audio: {e}")


Error loading audio: [WinError 2] Le fichier spécifié est introuvable


In [20]:
from pydub.utils import which

ffmpeg_path = which("ffmpeg")
print(f"FFmpeg found at: {ffmpeg_path}")


FFmpeg found at: C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\ffmpeg.exe


In [23]:
from pydub.utils import which
AudioSegment.ffmpeg = r"C:\ffmpeg\bin\ffmpeg.exe"  # Replace with your actual path to ffmpeg.exe


In [5]:
import os
from pydub.utils import which

# Manually set the path to FFmpeg to ensure correct version is used
os.environ["PATH"] = r"C:\ffmpeg\bin" + ";" + os.environ["PATH"]

# Now import AudioSegment after updating the path
from pydub import AudioSegment

# Ensure that the path is correctly detected
ffmpeg_path = which("ffmpeg")
print(f"FFmpeg found at: {ffmpeg_path}")

# Try loading the audio again
try:
    audio = AudioSegment.from_mp3(r"C:\AI PROJECTS\video_transcript\segments_temp\segment_0.mp3")
    print("Audio loaded successfully!")
except Exception as e:
    print(f"Error loading audio: {e}")



FFmpeg found at: C:\ffmpeg\bin\ffmpeg.exe
Audio loaded successfully!
